In [ ]:
%%HTML
<!-- Mejorar visualización en proyector -->
<style>
.rendered_html {font-size: 1.2em; line-height: 150%;}
div.prompt {min-width: 0ex; padding: 0px;}
.container {width:95% !important;}
</style>

In [1]:
%autosave 0
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display

Autosave disabled


# Problema de los valores/vectores propios

Sea una matriz cuadrada $A \in \mathbb{R}^{M\times M}$

El siguiente sistema de ecuaciones de $M$ ecuaciones

$$
(A - \lambda I) \vec v = 0,
$$

tiene como resultado $\lambda$, los valores propios de $A$ y $\vec v$ los vectores propios de $A$

La solución no trivial de este problema se obtiene de 

$$
|A - \lambda I | = 0
$$

que resulta en un polinomio de grado $M$ cuyas raices son $\lambda_i$, $i=1,2,\ldots, N$

Una vez determinado $\lambda_i$ se pueden usar para despejar $\vec v_i$

Podemos usar las funciones de [`scipy.linalg`](https://docs.scipy.org/doc/scipy/reference/linalg.html#eigenvalue-problems) `eig()` o `eigvals()` (y sus variantes) para resolver el sistema de ecuaciones

# Principal Component Analysis (PCA)

https://github.com/mlds-lab/COMPSCI-589/blob/master/slides/slides18.pdf

https://github.com/mlds-lab/COMPSCI-589/blob/master/slides/slides19.pdf

# Singular Value Decomposition (SVD)